In [ ]:
import time
import random
import requests
import pandas as pd

VS = "usd"
DAYS = 90

BASE_LIST   = "https://api.coingecko.com/api/v3/coins/list"
BASE_CHART  = "https://api.coingecko.com/api/v3/coins/{id}/market_chart"

session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (compatible; stable-demo/1.0)"
})

def fetch_all_coins():
    """全銘柄リスト取得"""
    r = session.get(BASE_LIST, timeout=30)
    r.raise_for_status()
    return r.json()   # [{id, symbol, name}, ...]

def fetch_market_chart(coin_id, days=90, vs="usd", max_retry=3):
    """指定コインの過去データ取得 (安全版・指数バックオフあり)"""
    url = BASE_CHART.format(id=coin_id)
    params = {"vs_currency": vs, "days": days}

    for attempt in range(max_retry):
        r = session.get(url, params=params, timeout=30)

        if r.status_code == 200:
            data = r.json()
            if "prices" not in data:
                raise RuntimeError(f"no 'prices' field: {coin_id}")
            df = pd.DataFrame(data["prices"], columns=["timestamp", "price"])
            df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
            return df

        if r.status_code == 429:
            wait_sec = 10 * (2 ** attempt)
            print(f"[429] waiting {wait_sec}s ... attempt {attempt+1}")
            time.sleep(wait_sec)
            continue

        # それ以外のエラーは raise
        r.raise_for_status()

    raise RuntimeError(f"Failed after retries: {coin_id}")

def main():
    # 1) 全銘柄リスト取得
    all_coins = fetch_all_coins()
    print(f"total coins available: {len(all_coins)}")

    # 2) ランダムに100銘柄選択
    sample_n = 100
    coins_sampled = random.sample(all_coins, sample_n)

    for idx, coin in enumerate(coins_sampled, start=1):
        coin_id = coin["id"]
        symbol  = coin["symbol"]
        name    = coin["name"]

        print(f"\n[{idx}/{sample_n}] Fetching {name} ({symbol}) id={coin_id}")

        try:
            df = fetch_market_chart(coin_id, DAYS, VS)
        except Exception as e:
            print(f"  -> error: {e}")
            continue

        # 3) CSV 保存
        safe_symbol = symbol.upper()
        safe_id = coin_id.replace("/", "_")
        filename = f"{idx:03d}_{safe_symbol}_{safe_id}_last{DAYS}d_{VS}.csv"

        df.to_csv(filename, index=False)
        print(f"  -> saved: {filename}")

        # 4) API制限対策：30秒待機（100銘柄で約50分）
        time.sleep(30)

if __name__ == "__main__":
    main()